# Claimable Liquidity from V3 Pool

In [ ]:
%load_ext credmark.cmf.ipython

param = {'chain_id': 1,
 'block_number': 15269596,
 'model_loader_path': ['../../credmark-models-py/models'],
 'chain_to_provider_url': {'1': 'http://192.168.68.122:10444'},
 'api_url': None,
 'use_local_models': '*',
 'register_utility_global': True}

context, model_loader = %cmf param

In [ ]:
load_from_ledger = False

pool = Contract(address='0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640')

if load_from_ledger:
    with pool.ledger.events.Mint as q:
        df_mint = q.select(columns=q.columns,
                        order_by=q.EVT_BLOCK_NUMBER.comma_(q.EVT_INDEX),
                        limit=5000
                        ).to_dataframe()

    with pool.ledger.events.Burn as q:
        df_burn = q.select(columns=q.columns,
                        order_by=q.EVT_BLOCK_NUMBER.comma_(q.EVT_INDEX),
                        limit=5000).to_dataframe()

    df_mint.to_csv('csv/uni_mint_5k.csv', index=False)
    df_burn.to_csv('csv/uni_burn_5k.csv', index=False)
else:
    df_mint = pd.read_csv('csv/uni_mint_5k.csv')
    df_burn = pd.read_csv('csv/uni_burn_5k.csv')


df_mint = df_mint.query('evt_block_number <= 12823596')
df_burn = df_burn.query('evt_block_number <= 12823596')

In [ ]:
df_mintburn = (pd
    .concat([df_mint.assign(type='mint', inp_amount = lambda x: x.inp_amount.apply(int)),
             df_burn.assign(type='burn', inp_amount = lambda x: -x.inp_amount.apply(int))])
    .sort_values(['evt_block_number', 'evt_index'])
    .reset_index(drop=True))

df_mintburn_idx = df_mintburn.set_index(['inp_tickLower', 'inp_tickUpper', 'inp_owner']).sort_index()

display(df_mintburn)
display(('Unique owners', df_mintburn.inp_owner.unique().tolist()))
display(('Unique sender', df_mintburn.inp_sender.unique().tolist()))

In [ ]:
def split_count(df):
    df_1 = df.query('inp_amount < 0').reset_index(drop=True)
    df_2 = df.query('inp_amount == 0').reset_index(drop=True)
    df_3 = df.query('inp_amount > 0').reset_index(drop=True)

    globals()['df_neg'] = df_1
    globals()['df_zero'] = df_2
    globals()['df_pos'] = df_3
    return df_1.shape[0], df_2.shape[0], df_3.shape[0]

(df_mintburn
    .groupby(['inp_tickLower', 'inp_tickUpper', 'inp_owner'], as_index=False)
    .inp_amount
    .sum()
    .pipe(split_count))

In [ ]:
sel = (199240, 199250, '0xC36442b4a4522E871399CD717aBDD847Ab11FE88')
display(df_mintburn_idx.loc[sel, :])

In [ ]:
from collections import namedtuple

Position = namedtuple('Position', 'lower upper owner')
# Detail = namedtuple('Detail', 'liquidity block_number')

df_mintburn_sel = df_mintburn.copy()

minted = {}
burnt = {}
live = {}

for n,r in df_mintburn_sel.iterrows():
    pos = Position(r.inp_tickLower, r.inp_tickUpper, r.inp_owner)
    detail = dict(liquidity=r.inp_amount, block_number=[r.evt_block_number], tx_hash=[r.evt_tx_hash])
    if r.type == 'mint':
        if pos in minted:
            minted[pos]['liquidity'] += detail['liquidity']
            minted[pos]['block_number'].extend(detail['block_number'])
            minted[pos]['tx_hash'].extend(detail['tx_hash'])
        else:
            minted[pos] = detail

        if pos in live:
            live[pos]['liquidity'] += detail['liquidity']
            live[pos]['block_number'] = detail['block_number']
            live[pos]['tx_hash'] = detail['tx_hash']
        else:
            live[pos] = detail.copy()
    else:
        if pos in burnt:
            burnt[pos]['liquidity'] += detail['liquidity']
            burnt[pos]['block_number'].extend(detail['block_number'])
            burnt[pos]['tx_hash'].extend(detail['tx_hash'])
        else:
            burnt[pos] = detail

        if pos in live:
            assert detail['block_number'][0] >= live[pos]['block_number'][0]
            liquidity_updated = live[pos]['liquidity'] + detail['liquidity']
            assert liquidity_updated >= -16384
            live[pos]['liquidity'] = liquidity_updated
            live[pos]['block_number'] = detail['block_number']
            live[pos]['tx_hash'] = detail['tx_hash']
        else:
            raise ValueError(f'{pos} not found in live')

In [ ]:
len([k for k,v in live.items() if v['liquidity'] < 0]), \
len([k for k,v in live.items() if v['liquidity'] == 0]), \
len([k for k,v in live.items() if v['liquidity'] > 0])

In [ ]:
(df_mintburn_idx.loc[(199270, 199280, '0xC36442b4a4522E871399CD717aBDD847Ab11FE88'),: ]
    .assign(csum=lambda x: x.inp_amount.cumsum().astype(float))
    .plot('evt_block_number', 'csum')
)

### Appendix: check L2 data

In [ ]:
with pool.ledger.events.Mint as q:
    mint_sel = q.select(columns=q.columns,
                    where=q.EVT_BLOCK_NUMBER.in_([12_730_140, 12_730_128])
                )
    df_mint_sel = mint_sel.to_dataframe()

with pool.ledger.events.Burn as q:
    burn_sel = q.select(columns=q.columns, # [q.EVT_BLOCK_NUMBER, q.AMOUNT],
                    where=q.EVT_BLOCK_NUMBER.in_([12_730_164])
                )
    df_burn_sel = burn_sel.to_dataframe()
display(df_mint_sel)
display(df_burn_sel)

### Appendix: get event from node

In [ ]:
df_burn_evt = (
    pd.DataFrame(pool.fetch_events(pool.events.Mint,
                       from_block=12730128,
                       to_block=12730128))
    .sort_values(['blockNumber', 'logIndex'])
    )
df_burn_evt

In [ ]:
df_burn_evt = (
    pd.DataFrame(pool.fetch_events(pool.events.Mint,
                       from_block=12730140,
                       to_block=12730140))
    .sort_values(['blockNumber', 'logIndex'])
    )
df_burn_evt

In [ ]:
df_burn_evt = (
    pd.DataFrame(pool.fetch_events(pool.events.Burn,
                       from_block=12730164,
                       to_block=12730164))
    .sort_values(['blockNumber', 'logIndex'])
    )
df_burn_evt

In [ ]:
df_burn_evt = (
    pd.DataFrame(pool.fetch_events(pool.events.Burn,
                       from_block=12706340,
                       to_block=12706340))
    .sort_values(['blockNumber', 'logIndex']))
df_burn_evt